In [1]:
# Method to access the transaction data of the routing node: RTL or gRPC
data_access_method = "gRPC"
# data_access_method = "RTL"

if (data_access_method == "gRPC"):

    # Replace with your routing node's certificate, macaroon file path, server port and protobuf file location
    cert_file = r'D:\lnd\node1\tls.cert'
    macaroon_file = r'D:\lnd\node1\data\chain\bitcoin\regtest\admin.macaroon'
    rpcserver = 'localhost:10009'
    protobuf_file_location = r'D:\lnd\proto\lnd\lnrpc' # Location of generated Protobuf files
    
    import sys
    # Ensure that this is the same version of Python used to generate the protobuf files
    print(sys.executable)
    print(sys.version)
    
    sys.path.append('D:\lnd\proto\lnd\lnrpc')
    
    import grpc
    import os
    from google.protobuf.json_format import MessageToDict
    from lightning_pb2 import ForwardingHistoryRequest
    from lightning_pb2 import ChannelGraphRequest
    from lightning_pb2 import GetInfoRequest
    from lightning_pb2_grpc import LightningStub
    
elif (data_access_method == "RTL"):
    
    # Replace with the name of the Ride The Lightning Forwarding History CSV file
    RTL_file = "Forwarding-history-sample.csv"
    
else: 
    
    print("Unsupported data access method.", data_access_method, " Must be gRPC or RTL")
    sys.exit(1)
    

C:\Users\Duncan\anaconda3\envs\myenv\python.exe
3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]


In [2]:
import pandas as pd
import networkx as nx
import ipycytoscape

In [3]:
# Function to handle button click
def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        if method_dropdown.value == 1:
            # Here you could call your gRPC functions
            print(f"Running gRPC with cert: {grpc_cert_text.value}, macaroon: {grpc_macaroon_text.value}, server: {grpc_rpcserver_text.value}")
        elif method_dropdown.value == 2:
            # Here you could call your CSV functions
            print(f"Running CSV with file path: {csv_file_text.value}")
        else:
            print("Please select a valid method.")

In [4]:
# Function to update the interface based on the dropdown
def on_method_change(change):
    if change['new'] == 1:
        grpc_inputs = [grpc_cert_text, grpc_macaroon_text, grpc_rpcserver_text]
        for widget in grpc_inputs:
            display(widget)
    elif change['new'] == 2:
        display(csv_file_text)
    else:
        clear_output()
        print("Please select a method.")


In [5]:
# Function to create a gRPC channel with the LND node
def create_grpc_channel(cert_file, macaroon_file):
    with open(cert_file, 'rb') as f:
        cert = f.read()
    with open(macaroon_file, 'rb') as f:
        macaroon = f.read().hex()

    credentials = grpc.ssl_channel_credentials(cert)
    channel = grpc.secure_channel('localhost:10009', credentials)
    return channel, macaroon

In [6]:
# Function to get node information
def get_node_info(stub, macaroon):
    request = GetInfoRequest()
    metadata = [('macaroon', macaroon)]
    response = stub.GetInfo(request, metadata=metadata)
    return response

In [7]:
# Function to get forwarding history
def get_forwarding_history(stub, start_time, end_time, macaroon):
    # Create a request with the specified time range (in Unix timestamp)
    request = ForwardingHistoryRequest(start_time=start_time, end_time=end_time, num_max_events=10000)
    #request = ForwardingHistoryRequest(start_time=start_time, end_time=end_time)
    #request = ForwardingHistoryRequest()
    
    # Include the macaroon in the metadata
    metadata = [('macaroon', macaroon)]

    # Make the gRPC call
    response = stub.ForwardingHistory(request, metadata=metadata)
    
    # Convert response to dictionary
    events = [MessageToDict(event) for event in response.forwarding_events]
    
    # Return as pandas DataFrame
    return pd.DataFrame(events)


In [8]:
def get_network_graph(stub, macaroon):
    
    # Include the macaroon in the metadata
    metadata = [('macaroon', macaroon)]
    
    request = ChannelGraphRequest(include_unannounced=True)
    response = stub.DescribeGraph(request, metadata=metadata)
    return response

In [9]:
def create_node_id_to_alias_map(graph):
    node_id_to_alias = {}
    for node in graph.nodes:
        node_id_to_alias[node.pub_key] = node.alias
    return node_id_to_alias

In [10]:
def create_channel_id_to_node_ids_map(graph):
    channel_id_to_node_ids = {}
    for edge in graph.edges:
        channel_id_to_node_ids[str(edge.channel_id)] = (edge.node1_pub, edge.node2_pub)
    return channel_id_to_node_ids

In [11]:
def get_alias_for_channel(channel_id, channel_id_to_node_ids, node_id_to_alias, exclude_node_pub_key):
    node_ids = channel_id_to_node_ids.get(str(channel_id), None)
    if node_ids:
        # Filter out the excluded node alias and return the other alias
        aliases = [node_id_to_alias.get(node_id) for node_id in node_ids if node_id != exclude_node_pub_key]
        if aliases:
            return aliases[0]
    return None


In [12]:
# Load the RTL data from the CSV file
def load_data(filepath):
    try:
        # Attempt to read the CSV file
        return pd.read_csv(filepath, encoding='utf-8')
    except FileNotFoundError:
        # Handle the case where the file doesn't exist
        print(f"Error: The file {filepath} was not found.")
        return None
    except PermissionError:
        # Handle the case where the file exists but you don't have the permission to open it
        print(f"Error: Permission denied when trying to read the file {filepath}.")
        return None
    except pd.errors.EmptyDataError:
        # Handle the case where the file is empty
        print(f"Error: The file {filepath} is empty.")
        return None
    except pd.errors.ParserError:
        # Handle the case where the file has parsing issues
        print(f"Error: The file {filepath} does not appear to be a valid CSV or is improperly formatted.")
        return None
    except Exception as e:
        # Handle any other exception and print out a message
        print(f"An unexpected error occurred: {e}")
        return None

In [13]:
def add_nodes_and_edges(G, df):
    """Add nodes and edges to the graph G based on the dataframe df."""
    for index, row in df.iterrows():
        
        # Convert node names to strings to handle numerical aliases
        u = str(row['alias_in'])
        v = str(row['alias_out'])
        
        # Convert 'amt_out_msat' and 'fee_msat' to integers
        weight = int(row['amt_out_msat'])
        fee = int(row['fee_msat'])
        
        # Check if there's already an edge (i.e., a previous transaction between these channels)
        if G.has_edge(u, v):
            edge_data = G[u][v]
            edge_data['weight'] += weight
            edge_data['fees'].append(fee)
        else:
            G.add_edge(u, v, weight=weight, fees=[fee])
    
    # Average out the fees and compute total transactions
    total_transactions = sum(nx.get_edge_attributes(G, 'weight').values())
    for u, v, d in G.edges(data=True):
        d['avg_fee'] = sum(d['fees']) / len(d['fees'])
        d['normalized_fee'] = d['avg_fee'] / total_transactions * 10000

    return total_transactions


In [14]:
# Assign classes to nodes based on normalized transactions and fees
def assign_classes_to_nodes (cyto_graph, df):
    
    # Ensure numeric columns are of the correct type
    df['fee_msat'] = pd.to_numeric(df['fee_msat'], errors='coerce')
    df['amt_out_msat'] = pd.to_numeric(df['amt_out_msat'], errors='coerce')

    for node in cyto_graph.graph.nodes:
        node_alias = node.data['id']

        classes = []
        
        # # Calculate total fees and total transactions for each node
        total_fees = df.groupby('alias_out')['fee_msat'].sum()
        total_transactions_gr = df.groupby('alias_out')['amt_out_msat'].sum()

        max_transaction = total_transactions_gr.max()

        # Normalize the transactions
        normalized_transactions_node = (total_transactions_gr / max_transaction).to_dict()
        normalized_fees = (total_fees / total_transactions_gr * 10000).to_dict()

        # Assign fee classes
        if node_alias in normalized_fees:

            fee = normalized_fees[node_alias]
            if fee <= 0.1:
                classes.append('low_fee')
            elif fee <= 0.2:
                classes.append('medium_fee')
            else:
                classes.append('high_fee')
        else:
            classes.append('no_fee')

        # Assign transaction classes
        if node_alias in normalized_transactions_node:
            trans = normalized_transactions_node[node_alias]
            if trans <= 0.1:
                classes.append('tiny_transaction')
            elif trans <= 0.5:
                classes.append('small_transaction')
            elif trans <= 0.9:
                classes.append('medium_transaction')
            else:
                classes.append('large_transaction')

        # Combine classes with space
        node.classes = ' '.join(classes)

In [15]:
# Assign classes to edges based on normalized transaction volumes
def assign_classes_to_edges (cyto_graph, G, total_transactions):

    for edge in cyto_graph.graph.edges:

        u, v = edge.data['source'], edge.data['target']   
        d = G[u][v]

        normalized_transaction = d['weight']/total_transactions * 10000

        if (normalized_transaction < 1):
            edge.classes = 'tiny_volume'
        elif normalized_transaction < 10:
            edge.classes = 'small_volume'
        elif normalized_transaction < 100:
            edge.classes = 'medium_volume'
        else:
            edge.classes = 'large_volume'

In [16]:
# Set styles based on classes
def set_styles (cyto_graph):
    
    cyto_graph.set_style([
        {
            'selector': 'node.no_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'red'
            }
        },
        {
            'selector': 'node.low_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'orange'
            }
        },
        {
            'selector': 'node.medium_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'yellow'
            }
        },
        {
            'selector': 'node.high_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'green'
            }
        },
            {
            'selector': 'node.tiny_transaction',
            'style': {
                'width': '10px',
                'height': '10px'
            }
        },
        {
            'selector': 'node.small_transaction',
            'style': {
                'width': '20px',
                'height': '20px'
            }
        },
        {
            'selector': 'node.medium_transaction',
            'style': {
                'width': '30px',
                'height': '30px'
            }
        },
        {
            'selector': 'node.large_transaction',
            'style': {
                'width': '50px',
                'height': '50px'
            }
        },
        {
            'selector': 'edge.tiny_volume',
            'style': {
                'width': '0px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.small_volume',
            'style': {
                'width': '1px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.medium_volume',
            'style': {
                'width': '2px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.large_volume',
            'style': {
                'width': '3px',
                'line-color': 'black'
            }
        }
    ])


In [17]:
# Replace with your routing node's certificate, macaroon file path and server port
cert_file = r'D:\lnd\node1\tls.cert'
macaroon_file = r'D:\lnd\node1\data\chain\bitcoin\regtest\admin.macaroon'
rpcserver = 'localhost:10009'

channel, macaroon = create_grpc_channel(cert_file, macaroon_file)
#info = get_node_info(channel, macaroon)

stub = LightningStub(channel)

# Extract the public key of the routing node
node_info = get_node_info(stub, macaroon)
routing_node_pub_key = node_info.identity_pubkey

start_time = 1609459200  # Example start time
end_time = 1800239131    # Example end time

# Fetch forwarding history
df = get_forwarding_history(stub, start_time, end_time, macaroon)

# Fetch the graph and create the maps
graph = get_network_graph(stub, macaroon)

node_id_to_alias = create_node_id_to_alias_map(graph)
channel_id_to_node_ids = create_channel_id_to_node_ids_map(graph)

# Exclude the routing node itself from the network graph
exclude_node_pub_key = routing_node_pub_key

# When applying the aliases to the DataFrame
df['alias_in'] = df['chanIdIn'].apply(lambda x: get_alias_for_channel(x, channel_id_to_node_ids, node_id_to_alias, exclude_node_pub_key))
df['alias_out'] = df['chanIdOut'].apply(lambda x: get_alias_for_channel(x, channel_id_to_node_ids, node_id_to_alias, exclude_node_pub_key))

# ALign the column names returned from gRPC with the CSV column names
df = df.rename(columns={
'feeMsat': 'fee_msat',
'amtInMsat': 'amt_in_msat',
'amtOutMsat': 'amt_out_msat',
'timestampNs': 'timestamp_ns'
})

# Create a new Cytograph
G = nx.Graph()

# Check if the data was loaded successfully
if df is None:
    # If df is None, print an error message and exit
    print("Transaction data could not be loaded. Please check the file path and format or API connection.")
    sys.exit(1)

total_transactions = add_nodes_and_edges(G, df)

cyto_graph = ipycytoscape.CytoscapeWidget()
cyto_graph.graph.add_graph_from_networkx(G)

cyto_graph.set_layout(name='circle')

assign_classes_to_nodes (cyto_graph, df)
assign_classes_to_edges (cyto_graph, G, total_transactions)

set_styles (cyto_graph)

# Note: 
# Ensure that ipytoscape is enabled in Jupyter Lab or Jupyter Notebook, in a Conda shell run the following
# jupyter nbextension enable --py --sys-prefix ipycytoscape
# Start Jupyter with "jupyter notebook (or jupyter lab) --NotebookApp.iopub_data_rate_limit=1.0e10", in Powershell if datarate is exceeded
# This can happen with large numbers of transactions

cyto_graph

    

  

    
    


Graph Nodes: [last_update: 1700593290
pub_key: "021257f2734bdf2396af9c425498c9c68e7e48183dd99879a1cb394f6bf192ccf0"
alias: "Don\'t node what to make of it!"
color: "#68f443"
features {
  key: 0
  value {
    name: "data-loss-protect"
    is_required: true
    is_known: true
  }
}
features {
  key: 5
  value {
    name: "upfront-shutdown-script"
    is_known: true
  }
}
features {
  key: 7
  value {
    name: "gossip-queries"
    is_known: true
  }
}
features {
  key: 9
  value {
    name: "tlv-onion"
    is_known: true
  }
}
features {
  key: 12
  value {
    name: "static-remote-key"
    is_required: true
    is_known: true
  }
}
features {
  key: 14
  value {
    name: "payment-addr"
    is_required: true
    is_known: true
  }
}
features {
  key: 17
  value {
    name: "multi-path-payments"
    is_known: true
  }
}
features {
  key: 23
  value {
    name: "anchors-zero-fee-htlc-tx"
    is_known: true
  }
}
features {
  key: 27
  value {
    name: "shutdown-any-segwit"
    is_known: 

CytoscapeWidget(cytoscape_layout={'name': 'circle'}, cytoscape_style=[{'selector': 'node.no_fee', 'style': {'c…